In [1]:
import os
import numpy as np
from scipy import interpolate
import scipy.io 
import Rbeast as rb
from SLM_tools import *
import pickle
import re

In [2]:
path1 = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI\testing data\energy_distance_mu_0.2_run_num_1.csv"
path2 = r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\SA_UI\testing data\energy_distance_mu_0.2_run_num_2.csv"

In [3]:
data = SLM_tools.load_data(path1,2,'csv',False)
energy = data[0]
distance = data[1]
energy, distance, time_vec = SLM_tools.downsample(energy,distance,1000)
o, cp, mean_trend = SLM_tools.beast(energy)


INFO: To supress printing the parameers in beast(),      set print.options = 0 
INFO: To supress printing the parameers in beast_irreg(),set print.options = 0 
INFO: To supress printing the parameers in beast123(),   set extra.printOptions = 0  
INFO: To supress warning messages in beast(),            set quiet = 1 
INFO: To supress warning messages in beast_irreg(),      set quiet = 1 
INFO: To supress warning messages in beast123(),         set extra.quiet = 1  

#--------------------------------------------------#
#       Brief summary of Input Data                #
#--------------------------------------------------#
Data Dimension: One signal of length 50000
IsOrdered     : Yes, ordered in time
IsRegular     : Yes, evenly spaced at interval of  1 (unknown unit)
hasSeasonCmpnt: False | no periodic or seasonal component. The model Y=Trend+Error is fitted.
HasOutlierCmpt: False | If true, Y=Trend+Outlier+Error (experimental) is fitted instead of Y=Trend+Error 
Detrend       : False 

In [4]:
A = SLM_tools.segment_data(energy, distance,mean_trend,cp)

In [5]:
data2 = SLM_tools.load_data(path2,2,'csv',False)
energy2 = data2[0]
distance2 = data2[1]
energy2, distance2, time_vec = SLM_tools.downsample(energy2,distance2,1000)
o2, cp2, mean_trend2 = SLM_tools.beast(energy2)


INFO: To supress printing the parameers in beast(),      set print.options = 0 
INFO: To supress printing the parameers in beast_irreg(),set print.options = 0 
INFO: To supress printing the parameers in beast123(),   set extra.printOptions = 0  
INFO: To supress warning messages in beast(),            set quiet = 1 
INFO: To supress warning messages in beast_irreg(),      set quiet = 1 
INFO: To supress warning messages in beast123(),         set extra.quiet = 1  

#--------------------------------------------------#
#       Brief summary of Input Data                #
#--------------------------------------------------#
Data Dimension: One signal of length 50000
IsOrdered     : Yes, ordered in time
IsRegular     : Yes, evenly spaced at interval of  1 (unknown unit)
hasSeasonCmpnt: False | no periodic or seasonal component. The model Y=Trend+Error is fitted.
HasOutlierCmpt: False | If true, Y=Trend+Outlier+Error (experimental) is fitted instead of Y=Trend+Error 
Detrend       : False 

In [6]:
B = SLM_tools.segment_data(energy2, distance2,mean_trend2,cp2)

In [7]:
C = []
C.append(A)
C.append(B)

In [8]:
with open('extracted_features_mu_0.2.pkl', 'wb') as file:
    pickle.dump(C, file)

In [9]:
with open('extracted_features_mu_0.2.pkl', 'rb') as file:
    loaded_C = pickle.load(file)

In [10]:
loaded_C

[[array([[-62.35478047],
         [-56.97475421],
         [-54.01507133],
         [-54.79772727],
         [-57.56755477]]),
  array([[19.74291086],
         [14.81567145],
         [13.25544642],
         [12.87340967],
         [13.57567309]]),
  array([[-25.36819576],
         [ -4.81387939],
         [ -4.48507454],
         [ -1.27758838],
         [ -2.56831266]]),
  array([[ 6924.],
         [21164.],
         [29856.],
         [35356.],
         [43801.]]),
  array([[-1.42915806e-03],
         [-2.28357175e-04],
         [-5.99643848e-05],
         [ 1.01872048e-03],
         [ 9.83552273e-04]]),
  array([[0.],
         [0.],
         [0.],
         [0.],
         [0.]]),
  array([[nan],
         [nan],
         [nan],
         [nan],
         [nan]])],
 [array([[-55.42533091],
         [-58.39829107],
         [-56.02851234],
         [-56.14836179],
         [-65.05037874]]),
  array([[12.83582855],
         [15.9915792 ],
         [12.69493879],
         [12.47315934],
  

In [11]:
c_reduced = SLM_tools.post_beast_processing(loaded_C)

In [12]:
with open('c_reduced_mu_0.2.pkl', 'wb') as file:
    pickle.dump(c_reduced, file)

In [13]:
principal_components, score, latent = SLM_tools.pca(c_reduced,5)

In [14]:
with open('pca_score_mu_0.2.pkl', 'wb') as file:
    pickle.dump(c_reduced, file)

In [15]:
c_reduced

array([[-6.23547805e+01,  1.97429109e+01, -1.42915806e-03,
        -2.53681958e+01,  6.92400000e+03,             nan],
       [-5.69747542e+01,  1.48156714e+01, -2.28357175e-04,
        -4.81387939e+00,  2.11640000e+04,             nan],
       [-5.40150713e+01,  1.32554464e+01, -5.99643848e-05,
        -4.48507454e+00,  2.98560000e+04,             nan],
       [-5.47977273e+01,  1.28734097e+01,  1.01872048e-03,
        -1.27758838e+00,  3.53560000e+04,             nan],
       [-5.75675548e+01,  1.35756731e+01,  9.83552273e-04,
        -2.56831266e+00,  4.38010000e+04,             nan],
       [-5.54253309e+01,  1.28358285e+01,  4.02530804e-04,
        -1.81982487e+00,  9.89700000e+03,  3.40510000e+04],
       [-5.83982911e+01,  1.59915792e+01,  1.69854142e-03,
        -1.01188907e+01,  2.07810000e+04,  2.41540000e+04],
       [-5.60285123e+01,  1.26949388e+01,  5.20279100e-04,
        -1.20654119e-01,  2.75500000e+04,  1.32700000e+04],
       [-5.61483618e+01,  1.24731593e+01, -1.282

In [16]:
score

array([[ 4.86339363e+00, -3.38429765e-01, -2.99991444e-01,
         8.41583352e-02, -6.05340488e-02],
       [ 3.52153356e-01, -3.66028475e-01,  2.27339458e-01,
        -1.00610287e-01,  2.78226902e-01],
       [-1.00113191e+00, -5.64654521e-01,  1.19389684e-01,
         7.68664505e-01, -2.63032120e-02],
       [-1.61986790e+00,  4.81950700e-01, -1.61886511e-01,
         3.26959511e-01, -3.51067435e-03],
       [-1.06490953e+00,  8.25720256e-01, -1.32165874e+00,
        -2.47787860e-01, -4.62128978e-02],
       [-4.21478704e-01, -2.01079223e-01,  1.64121405e+00,
        -2.58539630e-01, -9.74625177e-02],
       [ 8.01637337e-01,  1.79752015e+00,  2.48443345e-01,
        -8.22556504e-04,  1.83819654e-02],
       [-1.08337491e+00,  2.86923528e-02,  2.05906021e-01,
        -3.12852261e-01, -2.97872829e-02],
       [-8.26421370e-01, -1.66369148e+00, -6.58755862e-01,
        -2.59169756e-01, -3.27982334e-02]])

In [19]:
a_reduced = SLM_tools.post_pca_processing(score, c_reduced,5)

In [18]:
a_reduced

array([[ 4.86339363e+00, -3.38429765e-01, -6.05340488e-02,
        -2.99991444e-01,  8.41583352e-02,             nan],
       [ 3.52153356e-01, -3.66028475e-01,  2.78226902e-01,
         2.27339458e-01, -1.00610287e-01,             nan],
       [-1.00113191e+00, -5.64654521e-01, -2.63032120e-02,
         1.19389684e-01,  7.68664505e-01,             nan],
       [-1.61986790e+00,  4.81950700e-01, -3.51067435e-03,
        -1.61886511e-01,  3.26959511e-01,             nan],
       [-1.06490953e+00,  8.25720256e-01, -4.62128978e-02,
        -1.32165874e+00, -2.47787860e-01,             nan],
       [-4.21478704e-01, -2.01079223e-01, -9.74625177e-02,
         1.64121405e+00, -2.58539630e-01,  3.40510000e+04],
       [ 8.01637337e-01,  1.79752015e+00,  1.83819654e-02,
         2.48443345e-01, -8.22556504e-04,  2.41540000e+04],
       [-1.08337491e+00,  2.86923528e-02, -2.97872829e-02,
         2.05906021e-01, -3.12852261e-01,  1.32700000e+04],
       [-8.26421370e-01, -1.66369148e+00, -3.279

In [20]:
with open('a_reduced_mu_0.2.pkl', 'wb') as file:
    pickle.dump(a_reduced, file)

In [22]:
a_reduced

array([[ 4.86339363e+00, -3.38429765e-01, -6.05340488e-02,
        -2.99991444e-01,  8.41583352e-02,             nan],
       [ 3.52153356e-01, -3.66028475e-01,  2.78226902e-01,
         2.27339458e-01, -1.00610287e-01,             nan],
       [-1.00113191e+00, -5.64654521e-01, -2.63032120e-02,
         1.19389684e-01,  7.68664505e-01,             nan],
       [-1.61986790e+00,  4.81950700e-01, -3.51067435e-03,
        -1.61886511e-01,  3.26959511e-01,             nan],
       [-1.06490953e+00,  8.25720256e-01, -4.62128978e-02,
        -1.32165874e+00, -2.47787860e-01,             nan],
       [-4.21478704e-01, -2.01079223e-01, -9.74625177e-02,
         1.64121405e+00, -2.58539630e-01,  3.40510000e+04],
       [ 8.01637337e-01,  1.79752015e+00,  1.83819654e-02,
         2.48443345e-01, -8.22556504e-04,  2.41540000e+04],
       [-1.08337491e+00,  2.86923528e-02, -2.97872829e-02,
         2.05906021e-01, -3.12852261e-01,  1.32700000e+04],
       [-8.26421370e-01, -1.66369148e+00, -3.279

In [24]:
idn = np.where(a_reduced[:, 5] != 0)

In [25]:
idn

(array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64),)

In [26]:
a_reduced[:, 5]

array([   nan,    nan,    nan,    nan,    nan, 34051., 24154., 13270.,
        6501.])

In [27]:
np.random.seed(42)

In [28]:
idn = np.where(a_reduced[:, 5] != 0)

In [29]:
idn

(array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64),)

In [31]:
mapx = a_reduced[idn, :]

In [33]:
mapx.shape

(1, 9, 6)

In [21]:
np.random.seed(42)
idn = np.where(a_reduced[:, 5] != 0)
mapx = a_reduced[idn, :]
mapx = np.hstack((mapx[:, 0:2], np.log(mapx[:, 3])))
median_fit_vec = np.median(mapx[:, 3], axis=0)  # line 54 matlab
min = np.min(mapx[:, 3], axis=0)
max = np.max(mapx[:, 3], axis=0)
xw = mapx[:, 3]
points = np.linspace(min, max, 1000)
f_x = scipy.stats.gaussian_kde(points)  # line 60 matlab
p_x = f_x[xw]
yo = np.sort(mapx[:, 3])
aop = int(np.ceil((0.16 * len(yo)))) - 1
std_fit = np.median(mapx[:, 3], axis=0) - yo[aop]
x = np.copy(mapx)
train_index = int(np.floor(0.6 * x.shape[0]))
validation_index = int(np.floor(0.8 * x.shape[0]))
size_validation_set = len(range(train_index, validation_index))
tfas_predict_mat = np.zeros((10, size_validation_set))
tfas_actually_mat = np.zeros((10, size_validation_set))
mean_error_mat = np.zeros((10, size_validation_set))

IndexError: index 6 is out of bounds for axis 1 with size 6